Set up all the bokeh modules

In [ ]:
import numpy as np

from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure

try:
    from jupyterlabutils import show_with_bokeh_server
except ModuleNotFoundError:
    from notebook_utils import show_with_bokeh_server

This is necessary to make the plot show up in the notebook

In [ ]:
output_notebook(INLINE, hide_banner=True)

Set up data

In [ ]:
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

Set up plot

In [ ]:
plot = figure(plot_height=400, plot_width=400, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

_ = plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

Set up widgets

In [ ]:
text = TextInput(title="title", value='my sine wave')
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)

Set up callbacks for changing widgets

In [ ]:
def update_title(attrname, old, new):
    plot.title.text = text.value

text.on_change('value', update_title)

def update_data(attrname, old, new):

    # Get the current slider values
    a = amplitude.value
    b = offset.value
    w = phase.value
    k = freq.value

    # Generate the new curve
    x = np.linspace(0, 4*np.pi, N)
    y = a*np.sin(k*x + w) + b

    source.data = dict(x=x, y=y)

for w in [offset, amplitude, phase, freq]:
    w.on_change('value', update_data)

The following is the code that makes things work in the notebook.  First is a function that builds the proxy url and passes it on to `bokeh.io.show` second is a factory function that makes the plot given a document.

> Note that currently you can't run the following cell twice in the same kernel.  Bokeh thinks that objects are being used in multiple documents.  It's possible that putting everything in a guarding class could make that work, but I'm not sure it's necessary.

In [ ]:
def make_sine_plot(doc):
    # Set up layouts and add to document
    inputs = widgetbox(text, offset, amplitude, phase, freq)
    layout = row(inputs, plot)
    doc.add_root(layout)

show_with_bokeh_server(make_sine_plot)